## Disclaimer & Credits

In this kernel, I have attempted to re-implement the code for the second chapter of **Aurélien Géron's** amazing book [Hands-on Machine Learning with Scikit-Learn, Keras and Tensorflow](https://github.com/ageron/handson-ml2). You can find his detailed jupyter notebooks for each chapter in the link mentioned before. This notebook is primarily a way for me to internalize the content shared in each chapter of the book, and I hope it is useful to you. 


**Note:** _The code and content here is contained in the notebooks linked above. I have done my best not to include anything present in his book but not present in the notebooks._

# End to End Machine Learning Project
Use the California census to dataset to build a model of housing prices in the state

In [ ]:
# imports
import os
import pandas as pd

## Read the data

In [ ]:
HOUSING_PATH=os.path.join("../input","california-housing-prices")

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path,"housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()

## Look at the structure of the data

Let's use the `head` method to take a look at the first 5 rows of the dataframe. Herein, each row represents one district in the dataset.

In [ ]:
housing.head()

There are 10 attributes, namely longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value and ocean_proximity. Alternatively, you can see them via the `columns` member of the dataframe

In [ ]:
housing.columns

Using the `info` method, we can get information about the total number of rows, each feature's type and the number of *non-null* values. Below, we can see that there are some null values for the `ocean_proximity` feature and that there are **20640** rows in the dataset. Further, the `ocean_proximity` feature is non-numeric

In [ ]:
housing.info()

Let's take a look at the distribution of the `ocean-proximity` feature using the `value_counts` methods since it is non-numeric

In [ ]:
housing["ocean_proximity"].value_counts()

For the other features, we can use the `describe()` method which will give statistical information for each feature such as *count,mean,std,min,max* and *percentile* information

In [ ]:
housing.describe()

Additionally, plotting the histogram for each numerical attribute helps get a quick feel of the data. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50,figsize=(20,15))
plt.show()

**Observations from the plots:**
1. Some of the attributes such as `median_income`, `housing_median_age` and `median_house_value` (our target variable!) are clipped to a range
2. Attributes have different scales
3. Many histograms are tail-heavy, i.e., the distribution extends more to the right of the median than the left


## Creating a Test Set
Let's split the total data into training and test sets using different methods and compare results

In [ ]:
import numpy as np
# Fix the random seed
np.random.seed(42)

### Standard Train Test Split

In [ ]:
# Implementing basic train-test split
def split_train_test_data(total_data, test_frac=0.2):
    # Shuffle the indices of the data randomly
    shuffled_indices = np.random.permutation(len(total_data))
    # Length of test set
    test_set_len = int(test_frac*len(total_data))
    # Test Indices
    test_indices = shuffled_indices[:test_set_len]
    # Train Indices
    train_indices = shuffled_indices[test_set_len:]
    
    return total_data.iloc[train_indices], total_data.iloc[test_indices]

In [ ]:
# My split data
train_set, test_set = split_train_test_data(housing)

In [ ]:
# Split using Scikit-learn
from sklearn.model_selection import train_test_split

sk_train_set, sk_test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# Comparing data splits
train_set.head()

In [ ]:
sk_train_set.head()

### Stable Train Test Split

In [ ]:
# A more sophisticated split method: Keep the test set consistent even when new data is added to the total set
# When there's new data added to the set, the test set will definitely have all of the original test data despite shuffling
from zlib import crc32

def test_set_check(data_id, test_frac):
    data_hash_value = crc32(np.int64(data_id)) & 0xffffffff
    return data_hash_value < test_frac * 2**32

In [ ]:
def split_by_id(total_data, test_frac, id_column):
    ids = total_data[id_column]
    ids_in_test_set = ids.apply(lambda id_: test_set_check(id_, test_frac))
    
    return total_data[~ids_in_test_set], total_data[ids_in_test_set]

In [ ]:
# Create an id field for the dataset

# Option 1) Add the index field
housing_with_id = housing.reset_index()
train_set_stable, test_set_stable = split_by_id(housing_with_id, 0.2, "index")

In [ ]:
train_set_stable.head()

In [ ]:
# Option 2) Create a more stable id using latitude and longitude
housing_with_id = housing
housing_with_id["id"] = housing["longitude"]*1000 + housing["latitude"]
train_set_stable, test_set_stable = split_by_id(housing_with_id, 0.2, "id")

In [ ]:
train_set_stable.head()

### Stratified Split

In [ ]:
# Create a income category to see how the income values are distributed
housing["income_cat"] = pd.cut(housing["median_income"], 
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels = [1, 2, 3, 4, 5])

housing["income_cat"].hist()

To ensure that the test set is representative of all categories in the whole dataset, we can use stratified sampling

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Now let's see if the data has been split according the true representation of categories in the dataset

In [ ]:
# Original data's distribution across categories
housing["income_cat"].value_counts()/len(housing)

In [ ]:
# Stratified test set's distribution across categories
strat_test_set["income_cat"].value_counts()/len(strat_test_set)

As can be seen above, the test set has roughly the same distribution of class categories as the training set. Now we can move on to inspecting the data for gaining insights

In [ ]:
# Drop the "income_cat" feature from the stratified datasets

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    set_.drop("id", axis=1, inplace=True)

## Discover and Visualize the Data to Gain Insights

In [ ]:
# Create a copy of the stratified data for analysis
housing = strat_train_set.copy()

### Visualizing Geographical Data
Let's plot the data based on the latitude and longitude feature of each sample to see how the homes are distributed

In [ ]:
housing.plot(kind="scatter", # Type of plot
             x="longitude",
             y="latitude",
             alpha=0.4, # Transparency to visualize data easier
             s=housing["population"]/100, # Scale the points by the population; Gives us a sense of density
             label="Population",
             figsize=(20,14),
             c="median_house_value", # Color code by the house value; Gives us a sense of where homes cost more
             cmap=plt.get_cmap("jet"), # Blue indicates low values and Red indicates high values
             colorbar=True)

plt.legend()

Preliminary insights:
1. High density areas are Bay Area, Los Angeles and San Diego amongst others
2. Housing prices seem to be related to the location of the home (near the ocean) and the density of population

### Looking for Correlations

In [ ]:
# Compute the correlation of the target variable (median housing price) with all other features
corr_matrix = housing.corr()

corr_matrix["median_house_value"].sort_values(ascending=False)

The house value seems to be heavily correlated to the median income and weakly negatively correlated with latitude

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]

# The scatter matrix shows this relationship visually
scatter_matrix(housing[attributes], figsize=(24,16))

In [ ]:
# Of these attributes, the correlation with median income is most interesting
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.3,figsize=(12,8))

There's a clear upward trend between the price and income.

### Adding a few features based on the correlation

In [ ]:
# These are some features recommended by the author and as can be seen below, the bedrooms_per_room feature could be useful
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"] 
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
# Recompute the Correlation
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

## Prepare the Data for Machine Learning Algorithms

In [ ]:
# Revert to the clean training set for the next steps
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels since this is our training set
housing_labels = strat_train_set["median_house_value"].copy()

### Data Cleaning
There are three strategies commonly pursued for data cleaning as shown below:

In [ ]:
# Extracting the rows with missing info
incomplete_rows = housing[housing.isnull().any(axis=1)]
incomplete_rows.head()

The `total_bedrooms` feature has some NaNs so let's use that to try out different cleaning strategies

In [ ]:
incomplete_rows.dropna(subset=["total_bedrooms"]) # Option 1: Drop the rows which have NaNs for this feature

In [ ]:
incomplete_rows.drop("total_bedrooms",axis=1).head() # Option 2: Drop the feature completely

In [ ]:
median_value = housing["total_bedrooms"].median()
incomplete_rows["total_bedrooms"].fillna(median_value,inplace=True) # Option 3: Fill with something like the median value of the feature
incomplete_rows.head()

If we are filling missing values, `scikit-learn`'s `Imputer` can be used instead. However, it works only with numeric attributes so the text attributes have to be removed

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [ ]:
housing_numeric = housing.drop("ocean_proximity", axis=1)

In [ ]:
imputer.fit(housing_numeric)

In [ ]:
# Let's look at the statistics computed by the imputer
imputer.statistics_

In [ ]:
# Does it match the statistics of the original data?
housing_numeric.median().values

In [ ]:
# Transform the values in the training set
housing_numeric_transformed = imputer.transform(housing_numeric) 

In [ ]:
housing_tr = pd.DataFrame(housing_numeric_transformed, columns=housing_numeric.columns,
                          index=housing_numeric.index)

In [ ]:
housing_tr.head()

### Handling Text & Categorical Attributes


In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

#### Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_enc = ordinal_encoder.fit_transform(housing_cat)
housing_cat_enc[:10]

In [ ]:
ordinal_encoder.categories_ # labels run from 0 to 4

#### One Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_one_hot = cat_encoder.fit_transform(housing_cat)
housing_cat_one_hot

### Custom Transformers in Scikit-Learn

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# index of columns we are interested in
rooms_idx, bedrooms_idx, population_idx, households_idx = 3, 4, 5, 6

# Needs 3 functions to be defined without *args or **kwargs
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    
    # Function 1
    def __init__(self, add_bedrooms_per_room=True): 
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    # Function 2
    def fit(self, X, y=None):
        return self # nothing to do here
    
    # Function 3 : Adding the features we tried out earlier
    def transform(self, X):
        rooms_per_household = X[:,rooms_idx]/X[:,households_idx]
        population_per_household = X[:, population_idx]/X[:, households_idx]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_idx]/X[:,rooms_idx]
            return np.c_[X, rooms_per_household,population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X,rooms_per_household,population_per_household]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
        

In [ ]:
housing_extra_attribs = pd.DataFrame(housing_extra_attribs,
                                     columns=list(housing.columns) + ["rooms_per_household", "population_per_household"],
                                     index=housing.index)

housing_extra_attribs.head()

### Transformation Pipelines
A pipeline allows us to put all the preprocessing steps in sequence and perform them in the right order

In [ ]:
# Putting together all the steps of data processing into a pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler # For feature scaling so that all numeric attributes are in the same range

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")), # First Do this
    ('attribs_adder', CombinedAttributesAdder()), # Next Do this
    ('std_scaler', StandardScaler()), # Finally do this
])

housing_numeric_tr = numeric_pipeline.fit_transform(housing_numeric)

In [ ]:
housing_numeric_tr

But what about the `ocean_proximity` which is non-numeric?

In [ ]:
from sklearn.compose import ColumnTransformer

numeric_attr = list(housing_numeric)
cat_attr = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", numeric_pipeline, numeric_attr), # One pipeline for numeric data
    ("cat", OneHotEncoder(), cat_attr), # One pipeline for categorical data
])

housing_prepared = full_pipeline.fit_transform(housing) # One pipeline to rule them all ;) 

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape, housing.shape # Note that the extra columns are the features we added plus 5 for one-hot encoding

## Select and Train a Model
Now that the data's prepared, let's use it to train some models and check performance

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Evaluate on a few training examples
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data) # Pre-process the sample in the same way 
print("Predictions: ", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels: ", list(some_labels))

#### Evaluate the Model

In [ ]:
from sklearn.metrics import mean_squared_error

# How good is the model on the training set? Look's like it's underfitting quite a bit
lin_train_preds = lin_reg.predict(housing_prepared)

lin_reg_mse = mean_squared_error(lin_train_preds, housing_labels)
lin_reg_rmse = np.sqrt(lin_reg_mse)
lin_reg_rmse

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

#### Evaluate the Model

In [ ]:
tree_train_preds = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(tree_train_preds, housing_labels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse # The Decision Tree is clearly overfitting!

### Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

#### Evaluate the Model

In [ ]:
forest_train_preds = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(forest_train_preds, housing_labels)
forest_rmse = np.sqrt(forest_mse)
forest_rmse # Random Forest seems a lot better!

## Fine Tune your Model

### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())
    
display_scores(tree_rmse_scores) 

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                              scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores) # Decision Tree performs worse than Linear Regression!

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

### Grid Search 
Find the best hyperparameters for the problem by trying out all combinations that you specify

In [ ]:
from sklearn.model_selection import GridSearchCV

# Specify the parameters you are interesting in trying out
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, # Try a 3x4 combination first
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}, # Then try a 2x3 combination with bootstrapping set to False
]

forest_reg = RandomForestRegressor(random_state=42)

# Train Across 5 folds so totally (12 + 6) * 5 = 90 rounds of training
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring="neg_mean_squared_error",
                          return_train_score=True,
                          n_jobs=-1)

grid_search.fit(housing_prepared, housing_labels)


The best hyperparameter combination can be found as:

In [ ]:
grid_search.best_params_

And the best estimator as:

In [ ]:
grid_search.best_estimator_

So what are the scores of each combination we tested?

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

### Randomized Search
Give a range for each hyperparameter and it tries out a random combination of each hyperparameter by choosing random values for each one within the range

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'n_estimators': randint(low=1, high=200),
    'max_features': randint(low=1, high=8 ),
}

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                               n_iter=10, cv=5, scoring="neg_mean_squared_error", # Note that we've asked it to run 10 times
                               random_state=42, n_jobs=-1)
rnd_search.fit(housing_prepared, housing_labels)

Similarly, we can view the combination of cv scores and parameters that got them

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

### Analyze the Best Models and Their Errors
Let's take a look at the feature importances of the best model so far, i.e., the Random Forest

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = numeric_attr + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True) # Let's look at the features in their order of importance

### Evaluate the Model on the Test Set
Let's see how our optimized model does on the data that actually matters!

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
Y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(final_predictions, Y_test)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
# Let's compute a 95% confidence interval on our result
from scipy import stats

confidence=0.95
squared_errors = (final_predictions - Y_test)**2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, 
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))


## Exercises


### 1. Try a Support Vector Machine Regressor (SVR) with various hyperparameters. How does the best SVR predictor perform?


In [ ]:

from sklearn.svm import SVR

param_grid_svm = [
    {'kernel': ['linear'], 'C': [10., 30., 100.]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30.], 
     'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},    
]

svm_reg = SVR()
grid_search_svm = GridSearchCV(svm_reg, param_grid_svm, cv=5, scoring="neg_mean_squared_error", verbose=2, n_jobs=-1)
grid_search_svm.fit(housing_prepared, housing_labels)


In [ ]:

svm_mse = grid_search_svm.best_score_
svm_rmse = np.sqrt(-svm_mse)
svm_rmse # Looks much worse than the Random Forest Regressor


In [ ]:

# Best Parameters for SVM Regressor
grid_search_svm.best_params_


### 2. Replace GridSearchCV with RandomizedSearchCV


In [ ]:

from scipy.stats import expon, reciprocal

param_distribs_svm = {
    'kernel': ['linear', 'rbf'],
    'C': reciprocal(20, 2000),
    'gamma': expon(scale=1.0),
}

svm_reg = SVR()
rnd_search_svm = RandomizedSearchCV(svm_reg, param_distributions=param_distribs_svm,
                                n_iter=26, cv=5, scoring='neg_mean_squared_error', 
                                verbose=2, random_state=42, n_jobs=-1)
rnd_search_svm.fit(housing_prepared, housing_labels)


In [ ]:

svm_mse = rnd_search_svm.best_score_
svm_rmse = np.sqrt(-svm_mse)
svm_rmse # Seems a lot better right?


In [ ]:

rnd_search_svm.best_params_


### 3. Try adding a transformer in the preparation pipeline to select only the most important attributes
Let's first extract the "Top K" indices and then incorporate that into a custom transformer

In [ ]:
# Let's see if we still have this data from the Random Forest we ran a while back
feature_importances

In [ ]:
# Method to extract indices of the top-K most important features
def extract_top_k_indices(importances, k):
    # Get the indices we care about
    return np.sort(np.argpartition(importances,-k)[-k:]) 

In [ ]:
k=5
top_k_indices = extract_top_k_indices(feature_importances, k)
top_k_indices

In [ ]:
feature_names = np.array(attributes)[top_k_indices]
feature_names

In [ ]:
sorted(zip(feature_importances, attributes), reverse=True)[:k]

We have the right features selected, so let's move on

In [ ]:
class TopKFeatureSelector(BaseEstimator, TransformerMixin):
    # function 1
    def __init__(self, feature_importances, k):
        self.k = k
        self.feat_imp = feature_importances
    # function 2
    def fit(self, X, y=None):
        self.top_k_indices = extract_top_k_indices(self.feat_imp, self.k)
        return self
    # function 3
    def transform(self, X):
        return X[:, self.top_k_indices]

In [ ]:
feat_select_pipeline = Pipeline([
    ('preparation', full_pipeline), # same one as before
    ('feature_selection', TopKFeatureSelector(feature_importances, k)) # New block 
])

In [ ]:
housing_top_k_prepared = feat_select_pipeline.fit_transform(housing)
housing_top_k_prepared[:3] 

In [ ]:
housing_prepared[0:3, top_k_indices] # Check to see if we got it right

### 4. Try creating a single pipeline that does the full data preparation plus the final prediction

In [ ]:
end_to_end_pipeline = Pipeline([
    ('preparation', full_pipeline), # Prepare the Data
    ('feature_selection', TopKFeatureSelector(feature_importances, k)), # Choose the best features
    ('svm_reg', SVR(**rnd_search_svm.best_params_)) # Make Predictions
])

In [ ]:
end_to_end_pipeline.fit(housing, housing_labels)

In [ ]:
sample_data = housing.iloc[:5]
sample_labels = housing_labels.iloc[:5]

print("Predictions: ", end_to_end_pipeline.predict(sample_data))
print("True Labels: ", list(sample_labels))

### 5. Automatically explore some preparation options using GridSearchCV

In [ ]:
param_grid_auto = [{
    'preparation__num__imputer__strategy': ['median', 'most_frequent']
}]

auto_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('svm_reg', SVR(**rnd_search_svm.best_params_))
])

grid_search_prep = GridSearchCV(auto_pipeline, param_grid_auto, cv=5,
                                scoring='neg_mean_squared_error', verbose=2)
grid_search_prep.fit(housing, housing_labels)

In [ ]:
grid_search_prep.best_params_